In [44]:
import pandas as pd

# Đọc dữ liệu từ tệp
file_path = 'data/air_quality_hanoi_2020 .csv'
data = pd.read_csv(file_path)

# Thông tin tổng quan về dữ liệu
data_info = data.info()  # Cấu trúc dữ liệu, số lượng giá trị thiếu
data_head = data.head()  # Xem trước 5 dòng đầu
data_description = data.describe(include='all')  # Thống kê cơ bản

# Hiển thị thông tin
print(data_info)
print(data_head)
print(data_description)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         388 non-null    object 
 1   aqi          219 non-null    float64
 2   co           388 non-null    float64
 3   dew          369 non-null    float64
 4   humidity     369 non-null    float64
 5   no2          372 non-null    float64
 6   o3           310 non-null    float64
 7   pm10         388 non-null    float64
 8   pm25         388 non-null    float64
 9   pressure     369 non-null    float64
 10  so2          388 non-null    float64
 11  temperature  369 non-null    float64
 12  wind gust    3 non-null      float64
 13  wind speed   74 non-null     float64
 14  wind-gust    22 non-null     float64
 15  wind-speed   300 non-null    float64
dtypes: float64(15), object(1)
memory usage: 48.6+ KB
None
         Date    aqi   co   dew  humidity   no2   o3  pm10   pm25  pressure  \
0

In [45]:
# Bước 1: Xử lý các cột trùng lặp
# Kết hợp các cột gió ('wind gust', 'wind-gust') và ('wind speed', 'wind-speed')
data['wind_gust_combined'] = data[['wind gust', 'wind-gust']].mean(axis=1, skipna=True)
data['wind_speed_combined'] = data[['wind speed', 'wind-speed']].mean(axis=1, skipna=True)

# Loại bỏ các cột gốc không còn cần thiết
data_cleaned = data.drop(columns=['wind gust', 'wind-gust', 'wind speed', 'wind-speed'])

# Bước 2: Xử lý dữ liệu bị thiếu

# Điền giá trị trung bình cho các cột liên tục còn lại (dành cho các cột còn thiếu giá trị)
for column in ['aqi', 'dew', 'humidity', 'no2', 'o3', 'pressure', 'temperature', 
               'wind_gust_combined', 'wind_speed_combined']:
    if column in data_cleaned.columns:
        data_cleaned.loc[:, column] = data_cleaned[column].fillna(data_cleaned[column].mean())

# Bước 3: Xử lý định dạng dữ liệu
# Chuyển cột `Date` thành kiểu datetime
data_cleaned.loc[:, 'Date'] = pd.to_datetime(data_cleaned['Date'], format='%d/%m/%Y')

# Loại bỏ các cột không cần thiết
data_cleaned = data_cleaned.drop(columns=['Date'])

# Bước 4: Lưu dữ liệu sạch
output_file = 'data/cleaned_air_quality_data.csv'
data_cleaned.to_csv(output_file, index=False)

print(f"Dữ liệu sạch đã được lưu tại: {output_file}")


Dữ liệu sạch đã được lưu tại: data/cleaned_air_quality_data.csv


In [47]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# Tải dữ liệu sạch
cleaned_data = pd.read_csv('data/cleaned_air_quality_data.csv')

# Tách dữ liệu thành đầu vào (X) và đầu ra (y)
X = cleaned_data.drop(columns=['pm25'])  # Đầu vào
y = cleaned_data['pm25']  # Đầu ra: nồng độ bụi mịn PM2.5

# Khởi tạo danh sách các mô hình và tham số cho GridSearchCV
param_grids = {
    "Linear Regression": {
        "model": LinearRegression(),
        "params": {}
    },
    "Random Forest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {"n_estimators": [50, 100, 200], "max_depth": [5, 10, None], "min_samples_split": [2, 5, 10]}
    },
    "Support Vector Regressor": {
        "model": SVR(),
        "params": {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}
    },
    "Decision Tree": {
        "model": DecisionTreeRegressor(random_state=42),
        "params": {"max_depth": [3, 5, 10, None], "min_samples_split": [2, 5, 10]}
    },
    "K-Nearest Neighbors": {
        "model": KNeighborsRegressor(),
        "params": {"n_neighbors": [3, 5, 10], "weights": ["uniform", "distance"]}
    },
    "MLP Regressor": {
        "model": MLPRegressor(random_state=42, max_iter=1000),
        "params": {"hidden_layer_sizes": [(50,), (100,), (50, 50)], "alpha": [0.0001, 0.001, 0.01]}
    },
    "XGBoost Regressor": {
        "model": XGBRegressor(random_state=42),
        "params": {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1, 0.2], "max_depth": [3, 5, 7]}
    },
    "Lasso Regression": {
        "model": Lasso(random_state=42),
        "params": {"alpha": [0.01, 0.1, 1, 10]}
    },
    "Ridge Regression": {
        "model": Ridge(random_state=42),
        "params": {"alpha": [0.01, 0.1, 1, 10]}
    }
}

# Cài đặt K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Huấn luyện và tối ưu hóa các mô hình với GridSearchCV
results = []

for name, cfg in param_grids.items():
    print(f"Tuning {name}...")
    grid_search = GridSearchCV(cfg["model"], cfg["params"], scoring="neg_mean_squared_error", cv=kfold, n_jobs=-1)
    grid_search.fit(X, y)
    
    # Lấy mô hình tốt nhất
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_mse = -grid_search.best_score_  # Chuyển về giá trị MSE dương
    
    # Đánh giá trên tập kiểm tra
    y_pred = best_model.predict(X)
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    # Lưu kết quả đánh giá
    results.append({"Model": name, "Best Params": best_params, "MSE": mse, "MAE": mae, "R2": r2})

# Chuyển đổi kết quả đánh giá thành DataFrame
results_df = pd.DataFrame(results)

# Hiển thị kết quả đánh giá
print(results_df)


Tuning Linear Regression...
Tuning Random Forest...
Tuning Support Vector Regressor...
Tuning Decision Tree...
Tuning K-Nearest Neighbors...
Tuning MLP Regressor...
Tuning XGBoost Regressor...
Tuning Lasso Regression...
Tuning Ridge Regression...
                      Model  \
0         Linear Regression   
1             Random Forest   
2  Support Vector Regressor   
3             Decision Tree   
4       K-Nearest Neighbors   
5             MLP Regressor   
6         XGBoost Regressor   
7          Lasso Regression   
8          Ridge Regression   

                                         Best Params         MSE        MAE  \
0                                                 {}  187.062770  10.675049   
1  {'max_depth': 10, 'min_samples_split': 2, 'n_e...   29.733366   4.366126   
2                     {'C': 0.1, 'kernel': 'linear'}  192.913962  10.490254   
3          {'max_depth': 5, 'min_samples_split': 10}  142.768560   9.126904   
4          {'n_neighbors': 5, 'weights': 'dista

In [48]:
import joblib
import os

# Tạo thư mục 'models' nếu chưa tồn tại
os.makedirs("models", exist_ok=True)

# Lưu các mô hình tốt nhất
for result in results:
    model_name = result["Model"]
    best_model = param_grids[model_name]["model"]
    best_params = result["Best Params"]
    
    # Áp dụng tham số tốt nhất vào mô hình
    best_model.set_params(**best_params)
    best_model.fit(X, y)  # Huấn luyện lại mô hình với toàn bộ dữ liệu
    
    # Lưu mô hình vào thư mục 'models'
    model_file = f"models/{model_name.replace(' ', '_')}_optimized.joblib"
    joblib.dump(best_model, model_file)
    print(f"Saved {model_name} to {model_file}")


Saved Linear Regression to models/Linear_Regression_optimized.joblib
Saved Random Forest to models/Random_Forest_optimized.joblib
Saved Support Vector Regressor to models/Support_Vector_Regressor_optimized.joblib
Saved Decision Tree to models/Decision_Tree_optimized.joblib
Saved K-Nearest Neighbors to models/K-Nearest_Neighbors_optimized.joblib
Saved MLP Regressor to models/MLP_Regressor_optimized.joblib
Saved XGBoost Regressor to models/XGBoost_Regressor_optimized.joblib
Saved Lasso Regression to models/Lasso_Regression_optimized.joblib
Saved Ridge Regression to models/Ridge_Regression_optimized.joblib


In [50]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import ace_tools_open as tools  # Sử dụng ace_tools_open để hiển thị bảng

# Vector input mẫu (cần điều chỉnh để phù hợp với số lượng đặc trưng của dữ liệu)
sample_input = [[150, 0.45, 12.5, 60, 25, 18, 55, 1012, 5, 28.4, 3.2, 2.8]]  # Ví dụ input

# Tải toàn bộ các mô hình đã lưu trong thư mục 'models'
model_files = [f for f in os.listdir("models") if f.endswith(".joblib")]

# Kết quả lưu trữ
sample_test_results = []

for model_file in model_files:
    # Tải mô hình
    model_path = os.path.join("models", model_file)
    model = joblib.load(model_path)
    
    # Dự đoán với input mẫu
    try:
        prediction = model.predict(sample_input)[0]  # Lấy kết quả dự đoán đầu tiên
        
        # Lưu kết quả
        sample_test_results.append({
            "Model": model_file.replace("_optimized.joblib", "").replace("_", " "),
            "Sample Prediction (PM2.5)": prediction
        })
    except Exception as e:
        # Ghi nhận lỗi nếu xảy ra
        sample_test_results.append({
            "Model": model_file.replace("_optimized.joblib", "").replace("_", " "),
            "Sample Prediction (PM2.5)": f"Error: {str(e)}"
        })

# Chuyển kết quả thành DataFrame
sample_results_df = pd.DataFrame(sample_test_results)

# Hiển thị kết quả với ace_tools_open
tools.display_dataframe_to_user(name="Sample Input Test Results", dataframe=sample_results_df)


Sample Input Test Results


c:\Users\tanhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
c:\Users\tanhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
c:\Users\tanhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
c:\Users\tanhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\tanhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegr

In [52]:
# File để lưu kết quả đánh giá
evaluation_results_file = "resources/model_evaluation_results.csv"

# Danh sách lưu kết quả đánh giá
evaluation_results = []

for model_file in model_files:
    # Tải mô hình
    model_path = os.path.join("models", model_file)
    model_name = model_file.replace("_optimized.joblib", "").replace("_", " ")
    model = joblib.load(model_path)
    
    # Dự đoán trên toàn bộ dữ liệu
    try:
        y_pred = model.predict(X)
        mse = mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)
        
        # Lưu kết quả đánh giá
        evaluation_results.append({
            "Model": model_name,
            "MSE": mse,
            "MAE": mae,
            "R²": r2
        })
    except Exception as e:
        # Ghi nhận lỗi nếu xảy ra
        evaluation_results.append({
            "Model": model_name,
            "MSE": "Error",
            "MAE": "Error",
            "R²": f"Error: {str(e)}"
        })

# Chuyển kết quả thành DataFrame và lưu vào CSV
evaluation_results_df = pd.DataFrame(evaluation_results)
evaluation_results_df.to_csv(evaluation_results_file, index=False)

print(f"Saved evaluation results to {evaluation_results_file}")


Saved evaluation results to resources/model_evaluation_results.csv


In [2]:
import pandas as pd
import numpy as np
import joblib
import shap
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt
import os

# Tải dữ liệu sạch
cleaned_data = pd.read_csv("data/cleaned_air_quality_data.csv")

# Tách dữ liệu đầu vào (X) và đầu ra (y)
X = cleaned_data.drop(columns=['pm25'])  # Đầu vào
y = cleaned_data['pm25']  # Đầu ra

# Đường dẫn các thư mục lưu tài nguyên
RESOURCES_DIR = "resources"
MODELS_DIR = "models"
os.makedirs(RESOURCES_DIR, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

# Lưu giá trị đầu vào (X) và đầu ra (y)
X_file = os.path.join(RESOURCES_DIR, "X_values.csv")
y_file = os.path.join(RESOURCES_DIR, "y_values.csv")
X.to_csv(X_file, index=False)
y.to_csv(y_file, index=False)
print(f"Saved input features (X) to {X_file}")
print(f"Saved target values (y) to {y_file}")

# Đọc danh sách tất cả các mô hình
model_files = [f for f in os.listdir(MODELS_DIR) if f.endswith(".joblib")]
model_names = [f.replace("_optimized.joblib", "").replace("_", " ") for f in model_files]

# Dự đoán, tính SHAP và PDP cho từng mô hình
for model_name, model_file in zip(model_names, model_files):
    model_path = os.path.join(MODELS_DIR, model_file)
    if os.path.exists(model_path):
        model = joblib.load(model_path)
        print(f"Processing model: {model_name}...")

        # Dự đoán và lưu kết quả
        try:
            y_pred = model.predict(X)
            predictions_file = os.path.join(RESOURCES_DIR, f"{model_name.replace(' ', '_')}_predictions.csv")
            predictions_df = pd.DataFrame({"Actual": y, "Predicted": y_pred})
            predictions_df.to_csv(predictions_file, index=False)
            print(f"Saved predictions for model '{model_name}' to {predictions_file}")
        except Exception as e:
            print(f"Failed to generate predictions for model '{model_name}': {str(e)}")

        # Tính toán và lưu SHAP values
        print(f"Calculating SHAP values for model '{model_name}'...")
        try:
            explainer = shap.KernelExplainer(model.predict, shap.kmeans(X, 10))  # KernelExplainer
            shap_values = explainer.shap_values(X)
            shap_file = os.path.join(RESOURCES_DIR, f"{model_name.replace(' ', '_')}_shap_values.npy")
            np.save(shap_file, shap_values)
            print(f"Saved SHAP values for model '{model_name}' to {shap_file}")
        except Exception as e:
            print(f"Failed to calculate SHAP values for model '{model_name}': {str(e)}")

        # Tính toán và lưu Partial Dependence Plot (PDP)
        print(f"Calculating PDP for model '{model_name}'...")
        for feature_idx, feature_name in enumerate(X.columns):
            try:
                fig, ax = plt.subplots(figsize=(8, 6))
                PartialDependenceDisplay.from_estimator(
                    model, X, features=[feature_idx], ax=ax
                )
                plt.title(f"Partial Dependence Plot for {feature_name} - Model: {model_name}")
                pdp_image_file = os.path.join(RESOURCES_DIR, f"pdp_{model_name.replace(' ', '_')}_{feature_name}.png")
                plt.savefig(pdp_image_file)
                plt.close()
                print(f"Saved PDP plot for feature '{feature_name}' of model '{model_name}' to {pdp_image_file}")
            except Exception as e:
                print(f"Failed to calculate PDP for feature '{feature_name}' of model '{model_name}': {str(e)}")

print("All models processed.")


Saved input features (X) to resources\X_values.csv
Saved target values (y) to resources\y_values.csv
Processing model: Decision Tree...
Saved predictions for model 'Decision Tree' to resources\Decision_Tree_predictions.csv
Calculating SHAP values for model 'Decision Tree'...


100%|██████████| 388/388 [00:29<00:00, 13.12it/s]


Saved SHAP values for model 'Decision Tree' to resources\Decision_Tree_shap_values.npy
Calculating PDP for model 'Decision Tree'...
Saved PDP plot for feature 'aqi' of model 'Decision Tree' to resources\pdp_Decision_Tree_aqi.png
Saved PDP plot for feature 'co' of model 'Decision Tree' to resources\pdp_Decision_Tree_co.png
Saved PDP plot for feature 'dew' of model 'Decision Tree' to resources\pdp_Decision_Tree_dew.png
Saved PDP plot for feature 'humidity' of model 'Decision Tree' to resources\pdp_Decision_Tree_humidity.png
Saved PDP plot for feature 'no2' of model 'Decision Tree' to resources\pdp_Decision_Tree_no2.png
Saved PDP plot for feature 'o3' of model 'Decision Tree' to resources\pdp_Decision_Tree_o3.png
Saved PDP plot for feature 'pm10' of model 'Decision Tree' to resources\pdp_Decision_Tree_pm10.png
Saved PDP plot for feature 'pressure' of model 'Decision Tree' to resources\pdp_Decision_Tree_pressure.png
Saved PDP plot for feature 'so2' of model 'Decision Tree' to resources\pdp

c:\Users\tanhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\inspection\_plot\partial_dependence.py:972: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim([min_val, max_val])


Saved PDP plot for feature 'wind_speed_combined' of model 'Decision Tree' to resources\pdp_Decision_Tree_wind_speed_combined.png
Processing model: K-Nearest Neighbors...
Saved predictions for model 'K-Nearest Neighbors' to resources\K-Nearest_Neighbors_predictions.csv
Calculating SHAP values for model 'K-Nearest Neighbors'...


...

Saved SHAP values for model 'Random Forest' to resources\Random_Forest_shap_values.npy
Calculating PDP for model 'Random Forest'...
Saved PDP plot for feature 'aqi' of model 'Random Forest' to resources\pdp_Random_Forest_aqi.png
Saved PDP plot for feature 'co' of model 'Random Forest' to resources\pdp_Random_Forest_co.png
Saved PDP plot for feature 'dew' of model 'Random Forest' to resources\pdp_Random_Forest_dew.png
Saved PDP plot for feature 'humidity' of model 'Random Forest' to resources\pdp_Random_Forest_humidity.png
Saved PDP plot for feature 'no2' of model 'Random Forest' to resources\pdp_Random_Forest_no2.png
Saved PDP plot for feature 'o3' of model 'Random Forest' to resources\pdp_Random_Forest_o3.png
Saved PDP plot for feature 'pm10' of model 'Random Forest' to resources\pdp_Random_Forest_pm10.png
Saved PDP plot for feature 'pressure' of model 'Random Forest' to resources\pdp_Random_Forest_pressure.png
Saved PDP plot for feature 'so2' of model 'Random Forest' to resources\pdp

100%|██████████| 388/388 [00:29<00:00, 13.14it/s]


Saved SHAP values for model 'Ridge Regression' to resources\Ridge_Regression_shap_values.npy
Calculating PDP for model 'Ridge Regression'...
Saved PDP plot for feature 'aqi' of model 'Ridge Regression' to resources\pdp_Ridge_Regression_aqi.png
Saved PDP plot for feature 'co' of model 'Ridge Regression' to resources\pdp_Ridge_Regression_co.png
Saved PDP plot for feature 'dew' of model 'Ridge Regression' to resources\pdp_Ridge_Regression_dew.png
Saved PDP plot for feature 'humidity' of model 'Ridge Regression' to resources\pdp_Ridge_Regression_humidity.png
Saved PDP plot for feature 'no2' of model 'Ridge Regression' to resources\pdp_Ridge_Regression_no2.png
Saved PDP plot for feature 'o3' of model 'Ridge Regression' to resources\pdp_Ridge_Regression_o3.png
Saved PDP plot for feature 'pm10' of model 'Ridge Regression' to resources\pdp_Ridge_Regression_pm10.png
Saved PDP plot for feature 'pressure' of model 'Ridge Regression' to resources\pdp_Ridge_Regression_pressure.png
Saved PDP plot fo

100%|██████████| 388/388 [01:04<00:00,  6.01it/s]


Saved SHAP values for model 'Support Vector Regressor' to resources\Support_Vector_Regressor_shap_values.npy
Calculating PDP for model 'Support Vector Regressor'...
Saved PDP plot for feature 'aqi' of model 'Support Vector Regressor' to resources\pdp_Support_Vector_Regressor_aqi.png
Saved PDP plot for feature 'co' of model 'Support Vector Regressor' to resources\pdp_Support_Vector_Regressor_co.png
Saved PDP plot for feature 'dew' of model 'Support Vector Regressor' to resources\pdp_Support_Vector_Regressor_dew.png
Saved PDP plot for feature 'humidity' of model 'Support Vector Regressor' to resources\pdp_Support_Vector_Regressor_humidity.png
Saved PDP plot for feature 'no2' of model 'Support Vector Regressor' to resources\pdp_Support_Vector_Regressor_no2.png
Saved PDP plot for feature 'o3' of model 'Support Vector Regressor' to resources\pdp_Support_Vector_Regressor_o3.png
Saved PDP plot for feature 'pm10' of model 'Support Vector Regressor' to resources\pdp_Support_Vector_Regressor_pm1